In [1]:
import os
import sys
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tqdm.auto import tqdm

In [ ]:

current_dir = os.getcwd()
print(f"03 Notebook Current working directory: {current_dir}") # DEBUG

project_root = os.path.abspath(os.path.join(current_dir, '..'))
print(f"03 Notebook Calculated project root: {project_root}") # DEBUG

if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"03 Notebook Added project root to sys.path: {project_root}") # DEBUG

data_dir = os.path.join(project_root, 'data')
print(f"03 Notebook Calculated data directory: {data_dir}") # DEBUG


if not os.path.exists(data_dir):
    try:
        os.makedirs(data_dir)
        print(f"Successfully created directory: {data_dir}")
    except OSError as e:
        print(f"ERROR: Failed to create data directory {data_dir}. Reason: {e}")
        # Consider raising the error if it's critical to stop here
else:
    print(f"Data directory already exists: {data_dir}")


03 Notebook Current working directory: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project\notebook
03 Notebook Calculated project root: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project
03 Notebook Added project root to sys.path: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project
03 Notebook Calculated data directory: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project\data
Data directory already exists: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project\data


In [ ]:
# Load preprocessed data 
file_to_load = os.path.join(data_dir, 'tweet_eval_sentiment_multiclass_preprocessed.csv')

print(f"03 Notebook Attempting to load data from: {file_to_load}")
if not os.path.exists(file_to_load):
    print(f"ERROR: File does not exist at {file_to_load}.")
    raise FileNotFoundError(f"Missing preprocessed data file: {file_to_load}")

df = pd.read_csv(file_to_load)

df['processed_text'].fillna('', inplace=True)
df['processed_text'] = df['processed_text'].astype(str)
df = df[df['processed_text'].str.strip() != ''].copy()

03 Notebook Attempting to load data from: c:\Users\asus\OneDrive\Desktop\projects\sentiment_analysis_project\data\tweet_eval_sentiment_multiclass_preprocessed.csv


C:\Users\asus\AppData\Local\Temp\ipykernel_19884\2003256488.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['processed_text'].fillna('', inplace=True)


In [4]:
sample_df = df.sample(n=1000, random_state=42).copy()
print(f"Using {len(sample_df)} samples for Hugging Face pipeline evaluation.")

Using 1000 samples for Hugging Face pipeline evaluation.


In [5]:
X_sample = sample_df['text'] 
y_sample = sample_df['sentiment']
hf_model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
classifier = pipeline('sentiment-analysis', model=hf_model_name)
print(f"using hugging face model: {hf_model_name}")

Device set to use cpu


using hugging face model: cardiffnlp/twitter-roberta-base-sentiment


In [6]:
hf_predictions = []
for text in tqdm(X_sample, desc="predicting with hugging face pipeline"):
    result = classifier(text)[0]
    predicted_label = result['label']
    
    label_map = {'LABEL_0':0, 'LABEL_1':1, 'LABEL_2': 2}
    hf_predictions.append(label_map.get(predicted_label, -1))
hf_raw_results = []

predicting with hugging face pipeline:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:

for text in tqdm(X_sample, desc="Predicting with Hugging Face Pipeline"):
    
    result = classifier(text)
    hf_raw_results.append(result[0])
    predicted_label_text = result[0]['label']

    
    if predicted_label_text.upper() == 'NEGATIVE':
        hf_predictions.append(0)
    elif predicted_label_text.upper() == 'POSITIVE':
        hf_predictions.append(1)
    else:
        
        hf_predictions.append(-1) 


Predicting with Hugging Face Pipeline:   0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:

# Filter out any unmappable predictions if they occurred
if -1 in hf_predictions:
    initial_len = len(y_sample)
    valid_indices = [i for i, pred in enumerate(hf_predictions) if pred != -1]
    y_sample = y_sample.iloc[valid_indices]
    hf_predictions = [hf_predictions[i] for i in valid_indices]
    print(f"Filtered out {initial_len - len(y_sample)} samples due to unmappable HF predictions.")


Filtered out 0 samples due to unmappable HF predictions.


In [9]:

# Ensure lengths match before evaluation (after filtering)
if len(y_sample) != len(hf_predictions):
    print(f"Warning: Length mismatch. y_sample: {len(y_sample)}, hf_predictions: {len(hf_predictions)}")


In [ ]:
# Evaluate the Hugging Face pipeline
print("\n--- Evaluation for Hugging Face Pipeline ---")
print(f"Accuracy: {accuracy_score(y_sample, hf_predictions):.4f}")

print(f"Precision: {precision_score(y_sample, hf_predictions, average='weighted', zero_division=0):.4f}")
print(f"Recall: {recall_score(y_sample, hf_predictions, average='weighted', zero_division=0):.4f}")
print(f"F1-Score: {f1_score(y_sample, hf_predictions, average='weighted', zero_division=0):.4f}")
print("\nClassification Report (Hugging Face):")

print(classification_report(y_sample, hf_predictions, target_names=['Negative', 'Neutral', 'Positive'], zero_division=0))




--- Evaluation for Hugging Face Pipeline ---
Accuracy: 0.8140
Precision: 0.8166
Recall: 0.8140
F1-Score: 0.8141

Classification Report (Hugging Face):
              precision    recall  f1-score   support

    Negative       0.79      0.81      0.80       194
     Neutral       0.79      0.84      0.82       478
    Positive       0.87      0.78      0.82       328

    accuracy                           0.81      1000
   macro avg       0.82      0.81      0.81      1000
weighted avg       0.82      0.81      0.81      1000



In [11]:
import pickle
models_dir = os.path.join(project_root, 'models')

hf_eval_data={
    'y_pred': hf_predictions,
    'y_true': y_sample
}

with open (os.path.join(models_dir, 'hf_eval_data.pkl'), 'wb') as f:
    pickle.dump(hf_eval_data, f)
print("Hugging Face evaluation data saved for app.py")

Hugging Face evaluation data saved for app.py
